# --- Day 5: Hydrothermal Venture ---
## Part 1
You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:
```
    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.
```
For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?


In [1]:
import numpy as np
import pandas as pd
import pyparsing as pp

In [2]:
single_row_dfs = []
with open("../data/day05.dat", "r") as IH:
    pattern = pp.Word(pp.nums).set_results_name("x1") + "," + pp.Word(pp.nums).set_results_name("y1")
    pattern += pp.Word(" -> ") + pp.Word(pp.nums).set_results_name("x2") + "," + pp.Word(pp.nums).set_results_name("y2")
    for line in IH:
        parsed = pattern.parse_string(line.strip())
        single_row_dfs.append(pd.DataFrame(dict(x1=int(parsed.x1), y1=int(parsed.y1), x2=int(parsed.x2), y2=int(parsed.y2)), index=pd.Index([0])))
coords = pd.concat(single_row_dfs).reset_index(drop=True)

In [3]:
vertical_lines = coords.loc[coords["x1"] == coords["x2"]]
horizontal_lines = coords.loc[coords["y1"] == coords["y2"]]
max_grid_side = max(horizontal_lines.max().max(), vertical_lines.max().max())+1

In [4]:
map_grid = np.zeros((max_grid_side, max_grid_side))

In [5]:
for i in horizontal_lines.index:
    line = horizontal_lines.loc[i, :]
    start, stop = sorted([line["x1"], line["x2"]])
    map_grid[line["y1"], slice(start, stop+1)] += 1

In [6]:
for i in vertical_lines.index:
    line = vertical_lines.loc[i, :]
    start, stop = sorted([line["y1"], line["y2"]])
    map_grid[slice(start, stop+1), line["x1"]] += 1

In [7]:
(map_grid >= 2).sum()

5092

## --- Part Two ---

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:
```
    An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
    An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.
```
Considering all lines from the above example would now produce the following diagram:
```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```
You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?


In [8]:
diagonal_lines = coords.loc[np.logical_and(coords["x1"] != coords["x2"], coords["y1"] != coords["y2"])]

In [9]:
for i in diagonal_lines.index:
    line = diagonal_lines.loc[i, :]
    x_step = 1 if line["x1"] <= line["x2"] else -1
    x_range = list(range(line["x1"], line["x2"]+x_step, x_step))
    y_step = 1 if line["y1"] <= line["y2"] else -1
    y_range = list(range(line["y1"], line["y2"]+y_step, y_step))
    for pid in range(len(x_range)):
        map_grid[y_range[pid], x_range[pid]] += 1

In [10]:
(map_grid >= 2).sum()

20484